For this challenge, you will need to choose a corpus of data from nltk or another source that includes categories you can predict and create an analysis pipeline that includes the following steps:

Data cleaning / processing / language parsing

Create features using two different NLP methods: For example, BoW vs tf-idf.

Use the features to fit supervised learning models for each feature set to predict the category outcomes.

Assess your models using cross-validation and determine whether one model performed better.

Pick one of the models and try to increase accuracy by at least 5 percentage points.

Write up your report in a Jupyter notebook. Be sure to explicitly justify the choices you make throughout, and submit it below.

In [2]:
import nltk
# Launch the installer to download "gutenberg" and "stop words" corpora.
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [13]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter

In [26]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text

In [27]:
from nltk.corpus import inaugural

In [28]:
inaugural.fileids()

['1789-Washington.txt',
 '1793-Washington.txt',
 '1797-Adams.txt',
 '1801-Jefferson.txt',
 '1805-Jefferson.txt',
 '1809-Madison.txt',
 '1813-Madison.txt',
 '1817-Monroe.txt',
 '1821-Monroe.txt',
 '1825-Adams.txt',
 '1829-Jackson.txt',
 '1833-Jackson.txt',
 '1837-VanBuren.txt',
 '1841-Harrison.txt',
 '1845-Polk.txt',
 '1849-Taylor.txt',
 '1853-Pierce.txt',
 '1857-Buchanan.txt',
 '1861-Lincoln.txt',
 '1865-Lincoln.txt',
 '1869-Grant.txt',
 '1873-Grant.txt',
 '1877-Hayes.txt',
 '1881-Garfield.txt',
 '1885-Cleveland.txt',
 '1889-Harrison.txt',
 '1893-Cleveland.txt',
 '1897-McKinley.txt',
 '1901-McKinley.txt',
 '1905-Roosevelt.txt',
 '1909-Taft.txt',
 '1913-Wilson.txt',
 '1917-Wilson.txt',
 '1921-Harding.txt',
 '1925-Coolidge.txt',
 '1929-Hoover.txt',
 '1933-Roosevelt.txt',
 '1937-Roosevelt.txt',
 '1941-Roosevelt.txt',
 '1945-Roosevelt.txt',
 '1949-Truman.txt',
 '1953-Eisenhower.txt',
 '1957-Eisenhower.txt',
 '1961-Kennedy.txt',
 '1965-Johnson.txt',
 '1969-Nixon.txt',
 '1973-Nixon.txt',
 '1

In [54]:
# Load and clean the data.
p == 0 
for speech in inaugural.fileids():
    speech = inaugural.raw(speech)
    p += 1


In [33]:
reagan81 = text_cleaner(reagan81)
reagan85 = text_cleaner(reagan85) 

###### Spacy Parse
nlp = spacy.load('en')
reagan81_doc = nlp(reagan81)
reagan85_doc = nlp(reagan85)

In [57]:
#nltk sents
reagan81_sents = nltk.sent_tokenize(reagan81)
reagan85_sents = nltk.sent_tokenize(reagan85)

reagan81_sents = [[sent, "Reagan81"] for sent in reagan81_sents]
reagan85_sents = [[sent, "Reagan85"] for sent in reagan85_sents]


sentences = pd.DataFrame(reagan81_sents + reagan85_sents) 
sentences.tail()


,0,1
246,That's our heritage; that is our song.,Reagan85
247,We sing it still.,Reagan85
248,"For all our problems, our differences, we are ...",Reagan85
249,And may He continue to hold us close as we fil...,Reagan85
250,God bless you and may God bless America.,Reagan85


##### Spacy Group into sentences.
reagan81_sents = [[sent, "Reagan81"] for sent in reagan81_doc.sents]
reagan85_sents = [[sent, "Reagan85"] for sent in reagan85_doc.sents]

##### Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(reagan81_sents + reagan85_sents)
sentences.head()

In [59]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df



In [60]:
# Set up the bags.
reagan81words = bag_of_words(reagan81)
reagan85words = bag_of_words(reagan85)

# Combine bags to create a set of unique words.
common_words = set(reagan81words + reagan85words)

AttributeError: 'str' object has no attribute 'is_punct'